In [16]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, GridSearchCV

In [2]:
train = pd.read_csv("abalone_train.csv",index_col = 0)
train

,Sex,Length,Diameter,Height,Whole weight,Whole weight.1,Whole weight.2,Shell weight,Rings
id,,,,,,,,,
0,F,0.550,0.430,0.150,0.7715,0.3285,0.1465,0.2400,11
1,F,0.630,0.490,0.145,1.1300,0.4580,0.2765,0.3200,11
2,I,0.160,0.110,0.025,0.0210,0.0055,0.0030,0.0050,6
3,M,0.595,0.475,0.150,0.9145,0.3755,0.2055,0.2500,10
4,I,0.555,0.425,0.130,0.7820,0.3695,0.1600,0.1975,9
...,...,...,...,...,...,...,...,...,...
90610,M,0.335,0.235,0.075,0.1585,0.0685,0.0370,0.0450,6
90611,M,0.555,0.425,0.150,0.8790,0.3865,0.1815,0.2400,9
90612,I,0.435,0.330,0.095,0.3215,0.1510,0.0785,0.0815,6


In [3]:
testdf = pd.read_csv("abalone_test.csv",index_col = 0)

In [4]:
################################ Random Forest #####################################################

In [7]:
# as our y is multi-class col == use le;

le =LabelEncoder()
y = le.fit_transform(train["Rings"])
le.classes_

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 29], dtype=int64)

In [5]:
# define X:
# as we can see "sex" col == > catergorical data ==> convert into numerical

X = train.drop(["Rings"],axis = 1)
# crete dumies:
dum_X = pd.get_dummies(X,drop_first = True)   
dum_X 

,Length,Diameter,Height,Whole weight,Whole weight.1,Whole weight.2,Shell weight,Sex_I,Sex_M
id,,,,,,,,,
0,0.550,0.430,0.150,0.7715,0.3285,0.1465,0.2400,False,False
1,0.630,0.490,0.145,1.1300,0.4580,0.2765,0.3200,False,False
2,0.160,0.110,0.025,0.0210,0.0055,0.0030,0.0050,True,False
3,0.595,0.475,0.150,0.9145,0.3755,0.2055,0.2500,False,True
4,0.555,0.425,0.130,0.7820,0.3695,0.1600,0.1975,True,False
...,...,...,...,...,...,...,...,...,...
90610,0.335,0.235,0.075,0.1585,0.0685,0.0370,0.0450,False,True
90611,0.555,0.425,0.150,0.8790,0.3865,0.1815,0.2400,False,True
90612,0.435,0.330,0.095,0.3215,0.1510,0.0785,0.0815,True,False


In [6]:
# create object :

rf = RandomForestRegressor()

In [17]:
kfold = KFold(shuffle = True, n_splits = 5, random_state = 24)

params = {"max_depth": [None, 3,4,5]}
gcv = GridSearchCV(rf, param_grid = params, cv = kfold, scoring = "neg_mean_squared_log_error")

gcv = gcv.fit(dum_X, y)
print(gcv.best_score_)
print(gcv.best_params_)

-0.02969359635321665
{'max_depth': None}


In [19]:
# create dummoes of test

dum_tst = pd.get_dummies(testdf,drop_first = True)

# find y_pred
y_pred = gcv.predict(dum_tst)
res = pd.DataFrame({"id": list(testdf.index), "Rings": y_pred})
res

,id,Rings
0,90615,9.03
1,90616,8.87
2,90617,8.92
3,90618,10.33
4,90619,6.89
...,...,...
60406,151021,5.67
60407,151022,7.78
60408,151023,10.84
60409,151024,11.84


In [22]:

(res['Rings'] < 0).any().any()

False

In [24]:
# make csv file:

res.to_csv("sbt_abalone_02.csv", index = False)